<h3 style="text-align: center;"><b>Школа глубокого обучения ФПМИ МФТИ</b></h3>

<h3 style="text-align: center;"><b>Домашнее задание. Generative adversarial networks</b></h3>



В этом домашнем задании вы обучите GAN генерировать лица людей и посмотрите на то, как можно оценивать качество генерации

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! mkdir ~/.kaggle
! cp '/content/drive/MyDrive/Different_Stuff/Kaggle/kaggle.json' ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [3]:
! kaggle datasets download tommykamaz/faces-dataset-small
! unzip faces-dataset-small.zip

Dataset URL: https://www.kaggle.com/datasets/tommykamaz/faces-dataset-small
License(s): Attribution 4.0 International (CC BY 4.0)
100% 3.99G/3.99G [00:59<00:00, 84.6MB/s]
100% 3.99G/3.99G [00:59<00:00, 71.6MB/s]
Archive:  faces-dataset-small.zip
  inflating: faces_dataset_small/00055.png  
  inflating: faces_dataset_small/00237.png  
  inflating: faces_dataset_small/00240.png  
  inflating: faces_dataset_small/00241.png  
  inflating: faces_dataset_small/00242.png  
  inflating: faces_dataset_small/00243.png  
  inflating: faces_dataset_small/00244.png  
  inflating: faces_dataset_small/00245.png  
  inflating: faces_dataset_small/00246.png  
  inflating: faces_dataset_small/00247.png  
  inflating: faces_dataset_small/00248.png  
  inflating: faces_dataset_small/00249.png  
  inflating: faces_dataset_small/00253.png  
  inflating: faces_dataset_small/00255.png  
  inflating: faces_dataset_small/00257.png  
  inflating: faces_dataset_small/00258.png  
  inflating: faces_dataset_small/0

In [4]:
! rm faces-dataset-small.zip

In [5]:
! pip install torchinfo

In [6]:
import os
import cv2
import random
import pickle
from pathlib import Path
from PIL import Image, ImageDraw
import torch
import torch.nn as nn
from torch import optim
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image, make_grid
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from torchinfo import summary
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from IPython.display import clear_output
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import LeaveOneOut
from sklearn.manifold import TSNE
%matplotlib inline

sns.set(style='darkgrid', font_scale=1.2)

## Часть 1. Подготовка данных (1 балл)

В качестве обучающей выборки возьмем часть датасета [Flickr Faces](https://github.com/NVlabs/ffhq-dataset), который содержит изображения лиц людей в высоком разрешении (1024х1024). Оригинальный датасет очень большой, поэтому мы возьмем его часть. Скачать датасет можно [здесь](https://www.kaggle.com/datasets/tommykamaz/faces-dataset-small?resource=download-directory) и  [здесь](https://drive.google.com/drive/folders/14H7LQWzd09SaM11oLfGtd455sWH2rmRW?usp=share_link)

Давайте загрузим наши изображения. Напишите функцию, которая строит DataLoader для изображений, при этом меняя их размер до нужного значения (размер 1024 слишком большой, поэтому мы рекомендуем взять размер 128 либо немного больше)

In [7]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.cuda.empty_cache()

link = '/content/drive/MyDrive/Deep_Learning_School/13_generative_models/'
data_dir = Path('/content/faces_dataset_small')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

image_files = sorted(list(data_dir.rglob('*.png')))
print('Device:', device)
print('Length:', len(image_files))

Device: cuda
Length: 3143


In [8]:
class FlickrDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_list = os.listdir(root_dir)

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, index):
        img_name = os.path.join(self.root_dir, self.image_list[index])
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)
        return image

def get_dataloader(root_dir, image_size, batch_size):
    """
    Builds dataloader for training data.
    Use tt.Compose and tt.Resize for transformations
    :param image_size: height and wdith of the image
    :param batch_size: batch_size of the dataloader
    :returns: DataLoader object
    """

    transform = transforms.Compose([
            transforms.Resize(image_size),
            transforms.CenterCrop(image_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.5, 0.5, 0.5),
                                 std=(0.5, 0.5, 0.5))
    ])

    dataset = FlickrDataset(root_dir=root_dir, transform=transform)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    return data_loader

In [9]:
image_size = 128
batch_size = 128
stats = ((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

data_loader = get_dataloader(data_dir, image_size, batch_size)

In [10]:
def denorm(tensors):
    return tensors * stats[1][0] + stats[0][0]

def show_images(images):
    fig, ax = plt.subplots(figsize=(12, 24))
    ax.set_xticks([])
    ax.set_yticks([])
    ax.imshow(make_grid(denorm(images.detach()[:batch_size]),
                        nrow=8).permute(1, 2, 0))
    plt.title('')

In [11]:
for images in data_loader:
    show_images(images)
    break

Output hidden; open in https://colab.research.google.com to view.

## Часть 2. Построение и обучение модели (2 балла)

Сконструируйте генератор и дискриминатор. Помните, что:
* дискриминатор принимает на вход изображение (тензор размера `3 x image_size x image_size`) и выдает вероятность того, что изображение настоящее (тензор размера 1)

* генератор принимает на вход тензор шумов размера `latent_size x 1 x 1` и генерирует изображение размера `3 x image_size x image_size`

In [12]:
class GaussianNoise(nn.Module):
    def __init__(self, std=0.25):
        super().__init__()
        self.std = std

    def forward(self, x):
        if self.training:
            x += torch.empty_like(x).normal_(std=self.std)
        return x

def initialize_weights(model):
    classname = model.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(model.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(model.weight.data, 1.0, 0.02)
        nn.init.constant_(model.bias.data, 0.0)

In [13]:
gen_feature_size = 64
dis_feature_size = 64
output_channels = 1
latent_size = 128
kernel_size = 4
channels = 3
num_epochs = 200
lr = 1e-4

In [14]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.latent_size = latent_size

        self.generator = nn.Sequential(
            self.transposeBlock(latent_size, gen_feature_size * 16,
                                kernel_size, stride=1, padding=0),
            self.transposeBlock(gen_feature_size * 16, gen_feature_size * 8,
                                kernel_size, stride=2, padding=1),
            self.transposeBlock(gen_feature_size * 8, gen_feature_size * 4,
                                kernel_size, stride=2, padding=1),
            self.transposeBlock(gen_feature_size * 4, gen_feature_size * 2,
                                kernel_size, stride=2, padding=1),
            self.transposeBlock(gen_feature_size * 2, gen_feature_size,
                                kernel_size, stride=2, padding=1),
            nn.ConvTranspose2d(in_channels=gen_feature_size, out_channels=channels,
                               kernel_size=kernel_size, stride=2, padding=1,
                               bias=False),
            nn.Tanh()
        )

    def transposeBlock(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels=in_channels,
                               out_channels=out_channels,
                               kernel_size=kernel_size,
                               stride=stride,
                               padding=padding,
                               bias=False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2, inplace=True)
        )

    def forward(self, x):
        return self.generator(x)

summary(Generator().to('cpu').apply(initialize_weights),
       (batch_size, latent_size, 1, 1))

Layer (type:depth-idx)                   Output Shape              Param #
Generator                                [128, 3, 128, 128]        --
├─Sequential: 1-1                        [128, 3, 128, 128]        --
│    └─Sequential: 2-1                   [128, 1024, 4, 4]         --
│    │    └─ConvTranspose2d: 3-1         [128, 1024, 4, 4]         2,097,152
│    │    └─BatchNorm2d: 3-2             [128, 1024, 4, 4]         2,048
│    │    └─LeakyReLU: 3-3               [128, 1024, 4, 4]         --
│    └─Sequential: 2-2                   [128, 512, 8, 8]          --
│    │    └─ConvTranspose2d: 3-4         [128, 512, 8, 8]          8,388,608
│    │    └─BatchNorm2d: 3-5             [128, 512, 8, 8]          1,024
│    │    └─LeakyReLU: 3-6               [128, 512, 8, 8]          --
│    └─Sequential: 2-3                   [128, 256, 16, 16]        --
│    │    └─ConvTranspose2d: 3-7         [128, 256, 16, 16]        2,097,152
│    │    └─BatchNorm2d: 3-8             [128, 256, 16, 16

In [15]:
class Discriminator(nn.Module):
    def __init__(self, std=0.1):
        super(Discriminator, self).__init__()
        self.std = std

        self.discriminator = nn.Sequential(
            self.convBlock(channels, dis_feature_size,
                           kernel_size, stride=2, padding=1),
            self.convBlock(dis_feature_size, dis_feature_size * 2,
                           kernel_size, stride=2, padding=1),
            self.convBlock(dis_feature_size * 2, dis_feature_size * 4,
                           kernel_size, stride=2, padding=1),
            self.convBlock(dis_feature_size * 4, dis_feature_size * 8,
                           kernel_size, stride=2, padding=1),
            self.convBlock(dis_feature_size * 8, dis_feature_size * 16,
                           kernel_size, stride=2, padding=1),
            #GaussianNoise(self.std),
            nn.Conv2d(dis_feature_size * 16, output_channels,
                      kernel_size, stride=1, padding=0, bias=False),
            nn.Flatten(),
            nn.Sigmoid()
        )

    def convBlock(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            #GaussianNoise(self.std),
            nn.Conv2d(in_channels=in_channels,
                      out_channels=out_channels,
                      kernel_size=kernel_size,
                      stride=stride,
                      padding=padding,
                      bias=False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2, inplace=True)
        )

    def forward(self, x):
        return self.discriminator(x)

summary(Discriminator().to('cpu').apply(initialize_weights),
       (batch_size, channels, image_size, image_size))

Layer (type:depth-idx)                   Output Shape              Param #
Discriminator                            [128, 1]                  --
├─Sequential: 1-1                        [128, 1]                  --
│    └─Sequential: 2-1                   [128, 64, 64, 64]         --
│    │    └─Conv2d: 3-1                  [128, 64, 64, 64]         3,072
│    │    └─BatchNorm2d: 3-2             [128, 64, 64, 64]         128
│    │    └─LeakyReLU: 3-3               [128, 64, 64, 64]         --
│    └─Sequential: 2-2                   [128, 128, 32, 32]        --
│    │    └─Conv2d: 3-4                  [128, 128, 32, 32]        131,072
│    │    └─BatchNorm2d: 3-5             [128, 128, 32, 32]        256
│    │    └─LeakyReLU: 3-6               [128, 128, 32, 32]        --
│    └─Sequential: 2-3                   [128, 256, 16, 16]        --
│    │    └─Conv2d: 3-7                  [128, 256, 16, 16]        524,288
│    │    └─BatchNorm2d: 3-8             [128, 256, 16, 16]        512

In [16]:
def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DataLoaderDevice():
    def __init__(self, dataloader, device):
        self.dataloader = dataloader
        self.device = device

    def __iter__(self):
        for batch in self.dataloader:
            yield to_device(batch, self.device)

    def __len__(self):
        return len(self.dataloader)

In [22]:
data_loader = DataLoaderDevice(data_loader, device)

In [27]:
discriminator = Discriminator().to(device).apply(initialize_weights)
generator = Generator().to(device).apply(initialize_weights)

Перейдем теперь к обучению нашего GANа. Алгоритм обучения следующий:
1. Учим дискриминатор:
  * берем реальные изображения и присваиваем им метку 1
  * генерируем изображения генератором и присваиваем им метку 0
  * обучаем классификатор на два класса

2. Учим генератор:
  * генерируем изображения генератором и присваиваем им метку 0
  * предсказываем дискриминаторором, реальное это изображение или нет


В качестве функции потерь берем бинарную кросс-энтропию

In [28]:
model = {
    "discriminator": discriminator,
    "generator": generator
}

criterion = {
    "discriminator": nn.BCELoss(),
    "generator": nn.BCELoss()
}

optimizer = {
    "discriminator": optim.Adam(discriminator.parameters(), lr),
    "generator": optim.Adam(generator.parameters(), lr)
}

scheduler = {
    "discriminator": optim.lr_scheduler.StepLR(optimizer['discriminator'], step_size=80, gamma=0.9),
    "generator": optim.lr_scheduler.StepLR(optimizer['generator'], step_size=80, gamma=0.9)
}

In [29]:
def fit(model, criterion, optimizer, scheduler, epochs, loaded=False):
    model['discriminator'].train()
    model['generator'].train()
    torch.cuda.empty_cache()

    dis_losses = []
    gen_losses = []
    real_scores = []
    fake_scores = []

    if loaded:
        dis_losses, gen_losses, real_scores, fake_scores = pickle.load(file)

    for epoch in range(epochs):
        dis_loss_per_epoch = []
        gen_loss_per_epoch = []
        real_score_per_epoch = []
        fake_score_per_epoch = []

        for real_images in tqdm(data_loader):
            # Train discriminator

            optimizer['discriminator'].zero_grad()
            real_predictions = model['discriminator'](real_images)
            real_labels = 0.1 * torch.rand(real_images.size(0), 1, device=device) + 0.9
            real_loss = criterion['discriminator'](real_predictions, real_labels)
            real_score_per_epoch.append(torch.mean(real_predictions).item())

            latent_vector = torch.randn(batch_size, latent_size, 1, 1, device=device)
            fake_images = model['generator'](latent_vector)
            fake_labels = 0.1 * torch.rand(fake_images.size(0), 1, device=device)
            fake_predictions = model['discriminator'](fake_images)
            fake_loss = criterion['discriminator'](fake_predictions, fake_labels)
            fake_score_per_epoch.append(torch.mean(fake_predictions).item())

            dis_loss = real_loss + fake_loss
            dis_loss.backward()
            optimizer['discriminator'].step()
            dis_loss_per_epoch.append(dis_loss.item())

            # Train generator

            optimizer['generator'].zero_grad()
            latent_vector = torch.randn(batch_size, latent_size, 1, 1, device=device)
            fake_images = model['generator'](latent_vector)
            predictions = model['discriminator'](fake_images)
            labels = torch.ones(batch_size, 1, device=device)

            gen_loss = criterion['generator'](predictions, labels)
            gen_loss.backward()
            optimizer['generator'].step()
            gen_loss_per_epoch.append(gen_loss.item())

        dis_losses.append(np.mean(dis_loss_per_epoch))
        gen_losses.append(np.mean(gen_loss_per_epoch))
        real_scores.append(np.mean(real_score_per_epoch))
        fake_scores.append(np.mean(fake_score_per_epoch))

        scheduler['discriminator'].step()
        scheduler['generator'].step()

        clear_output(wait=True)
        print("Epoch [{}/{}], dis_loss: {:.4f}, gen_loss: {:.4f}, real_score: {:.4f}, fake_score: {:.4f}" \
              .format(epoch + 1, epochs,
                      dis_losses[-1], gen_losses[-1],
                      real_scores[-1], fake_scores[-1]))

        if epoch % 20 == 0:
            torch.save(model["discriminator"].state_dict(), link + 'discriminator.pth')
            torch.save(model["generator"].state_dict(), link + 'generator.pth')
            lists = [dis_losses, gen_losses, real_scores, fake_scores]
            with open(link + 'lists.txt', 'wb') as file:
                pickle.dump(lists, file)

    return dis_losses, gen_losses, real_scores, fake_scores

Постройте графики лосса для генератора и дискриминатора. Что вы можете сказать про эти графики?

In [ ]:
trained = False

if not trained:
    dis_losses, gen_losses, real_scores, fake_scores = fit(model, criterion, optimizer, scheduler, num_epochs)

Epoch [43/200], dis_loss: 0.4297, gen_loss: 4.6797, real_score: 0.9428, fake_score: 0.0567


  0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
with open(link + 'lists.txt', rb) as file:
    dis_losses, gen_losses, real_scores, fake_scores = pickle.load(file)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 15))

ax1.plot(dis_losses)
ax1.plot(gen_losses)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend(['Discriminator', 'Generator'])
ax1.set_title('Losses')

ax2.plot(real_scores)
ax2.plot(fake_scores)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Score')
ax2.legend(['Real', 'Fake'])
ax2.set_title('Scores')

In [ ]:
model["discriminator"].eval()
model["generator"].eval()

## Часть 3. Генерация изображений (1 балл)

Теперь давайте оценим качество получившихся изображений. Напишите функцию, которая выводит изображения, сгенерированные нашим генератором

In [ ]:
n_images = 64

fixed_latent = torch.randn(n_images, latent_size, 1, 1, device=device)
fake_images = model["generator"](fixed_latent)

In [ ]:
def show_images(fake_images):
    fig, ax = plt.subplots(figsize=(15, 15))
    ax.set_xticks([])
    ax.set_yticks([])
    ax.imshow(make_grid(denorm(fake_images.cpu().detach()), nrow=8).permute(1, 2, 0))

In [ ]:
show_images(fake_images)

## Часть 4. Leave-one-out-1-NN classifier accuracy (6 баллов)

### 4.1. Подсчет accuracy (4 балла)

Не всегда бывает удобно оценивать качество сгенерированных картинок глазами. В качестве альтернативы вам предлагается реализовать следующий подход:
  * Сгенерировать столько же фейковых изображений, сколько есть настоящих в обучающей выборке. Присвоить фейковым метку класса 0, настоящим – 1.
  * Построить leave-one-out оценку: обучить 1NN Classifier (`sklearn.neighbors.KNeighborsClassifier(n_neighbors=1)`) предсказывать класс на всех объектах, кроме одного, проверить качество (accuracy) на оставшемся объекте. В этом вам поможет `sklearn.model_selection.LeaveOneOut`

In [ ]:
len_data = 1000
real_images = torch.FloatTensor()
for i in tqdm(range(len_data)):
    real_images = torch.cat((real_images, next(iter(data_loader)).cpu()))
real_one = torch.ones(len_data, 1, device=device)

with torch.no_grad():
    fake_images = model['generator'](torch.randn(len_data, latent_size,
                                                 1, 1, device=device))
fake_zero = torch.zeros(len_data, 1, device=device)

images = torch.cat((real_images.to('cpu'), fake_images.to('cpu')))
num_images = images.shape[0]
images = images.reshape(num_images, -1).numpy()
labels = torch.cat((real_one.to('cpu'), fake_zero.to('cpu'))).numpy().ravel()

In [ ]:
%%time
scores = cross_val_score(
    KNeighborsClassifier(n_neighbors=1, n_jobs=-1),
    images, labels,
    cv=LeaveOneOut()
)

In [ ]:
df = pd.DataFrame({
    'accuracy': accuracies,
    'label': classes
})

print(df['accuracy'].mean())
df.pivot_table(value='accuracy',
               index='label',
               aggfunc='mean')

Что вы можете сказать о получившемся результате? Какой accuracy мы хотели бы получить и почему?

### 4.2. Визуализация распределений (2 балла)

Давайте посмотрим на то, насколько похожи распределения настоящих и фейковых изображений. Для этого воспользуйтесь методом, снижающим размерность (к примеру, TSNE) и изобразите на графике разным цветом точки, соответствующие реальным и сгенерированным изображенияи

In [ ]:
tsne_images = TSNE(n_components=2).fit_transform(images)

plt.figure(figsize=(15, 8))
sns.scatterplot(x=tsne_images[:, 0], y=tsne_images[:, 1], hue=labels, palette='rocket')
plt.show()